In [1]:
import numpy as np
from numpy import random
from random import sample 
import scipy
from sklearn.metrics import pairwise_distances
from scipy.stats import rankdata
from scipy.stats import kendalltau
from scipy.stats import pearsonr
from scipy.stats import spearmanr
import time
from itertools import combinations 

In [2]:
from taupath import *

In [3]:
def compute_step_add(X, ind, state, states, method):
    row,col = X.shape
    to_add = list(set(range(row)) - set(ind))
    for add in to_add:
        new_ind = sorted(ind+[add])
        idx = ','.join([str(ii) for ii in new_ind])
        if idx in states:
            continue
        states[idx] = compute_score_add(X,ind,add,state,method)
        

In [4]:
def compute_step_del(X, ind, state, states, method):
    row,col = X.shape
    to_del = list(ind)
    for d in to_del:
        new_ind = ind.copy()
        new_ind.remove(d)
        idx = ','.join([str(ii) for ii in new_ind])
        if idx in states:
            continue
        states[idx] = compute_score_del(X,ind,d,state,method)

In [5]:
def forward_select(X,low,method):
    row,col = X.shape
    for step in range(3,low+1):
        states = dict()
        if step == 3:
            comb = combinations(range(row), step)
            for ind in comb:
                ind = sorted(ind)
                idx = ','.join([str(ii) for ii in ind])
                states[idx] = compute_score_initial(X,ind,method)
        else:
            for idx,state in cand.items():
                ind = [int(i) for i in idx.split(',')]
                compute_step_add (X, ind, state, states, method)
        score = [v.score for k,v in states.items()]
        best_score = max(score)   
        cand = dict()
        for k,v in states.items():
            if v.score==best_score: cand[k] = v
    for k,v in cand.items():
        print("Forward_Subpopulation:",k,"Best Score:",v.score)
    return cand         

In [6]:
def backward_select(X,low,method):
    row,col = X.shape
    h = row
    for step in range(h,low-1,-1):
        states = dict()
        if step == h:
            ind = list(range(h))
            idx = ','.join([str(ii) for ii in ind])
            states[idx] = compute_score_initial(X,ind,method)
        else:
            for idx,state in cand.items():
                ind = [int(i) for i in idx.split(',')]
                compute_step_del (X, ind, state, states, method)
        score = [v.score for k,v in states.items()]
        best_score = max(score)   
        cand = dict()
        for k,v in states.items():
            if v.score==best_score: cand[k] = v
        if step < low: break   
    for k,v in cand.items():
        print("Backward_Subpopulation:",k,"Best Score:",v.score)
    return cand

In [7]:
# li1 = [1,2,3]
# li2 = [2,3,4,5,]
def Diff(li2, li1):
    return (list(list(set(li1)-set(li2))))
#Diff(li2,li1)

# Simulation

# N=30, P=50

We compute True Positive ratio for each method

In [9]:
N = 30
for method in ['Pearson','Kendall']:
#     for prop in [0.3,0.5,0.8]:
#         for rho in [0.3,0.5,0.8]:
    for prop in [0.3]:
        for rho in [0.8]:
            n1 = int(N*prop)
            n2 = int(N*(1-prop))
            p = 50
            mean = [0]*p
#             cov = [[1, rho,rho,rho,rho], [rho, 1,rho,rho,rho],[rho,rho,1,rho,rho],[rho,rho,rho,1,rho],[rho,rho,rho,rho,1]] 
            cov = np.full((p, p), rho)
            np.fill_diagonal(cov, 1)
            q1 = np.random.multivariate_normal(mean, cov, n1)
            mean2 = [0]*p
            cov2 = np.identity(p)
            q2 = np.random.multivariate_normal(mean2, cov2, n2)
            q = np.concatenate((q1, q2), axis=0)
            low=n1
            print("propotion:",prop, "Rho:",rho,"Method:", method,"Sample1:",n1)
            f = forward_select(q,low,method)
            b = backward_select(q,low,method)
            ga_pop, ga_score = sub_pop_state(q,cand_size=1000,low=low,high=low+3,top=50,kp=100,rep=150,method=method)
            for k,v in f.items():
                a = list(eval(k))
                print('Forward_TT:',1-(len(Diff(a,range(n1)))/n1))

            for k,v in b.items():
                a = list(eval(k))
                print('Backward_TT:',1-(len(Diff(a,range(n1)))/n1))

            for k in ga_pop:
                print('GA_TT:',1-(len(Diff(k,range(n1)))/n1))


propotion: 0.3 Rho: 0.8 Method: Pearson Sample1: 9
Forward_Subpopulation: 0,1,2,4,5,6,7,8,19 Best Score: 0.5401814058956909
Backward_Subpopulation: 0,1,2,4,5,6,7,8,19 Best Score: 0.5401814058956909


/home/qguo0701/Projects/Tau-path/taupath.py:336: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  out = np.array(cand)[np.where([sc == score[0] for sc in score])[0]]


GA_Subpopulation: [[0, 1, 2, 4, 5, 6, 7, 8, 19]] Best Score: 0.5401814058956909
Forward_TT: 0.8888888888888888
Backward_TT: 0.8888888888888888
GA_TT: 0.8888888888888888
propotion: 0.3 Rho: 0.8 Method: Kendall Sample1: 9
Forward_Subpopulation: 0,1,2,3,4,5,6,7,8 Best Score: 0.7151927437641705
Backward_Subpopulation: 0,1,2,3,4,5,6,7,8 Best Score: 0.7151927437641705


/home/qguo0701/Projects/Tau-path/taupath.py:336: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  out = np.array(cand)[np.where([sc == score[0] for sc in score])[0]]


GA_Subpopulation: [[0, 1, 2, 3, 4, 5, 6, 7, 8]] Best Score: 0.7151927437641705
Forward_TT: 1.0
Backward_TT: 1.0
GA_TT: 1.0
